In [1]:
#read parameters from command
image_type = 'Liver'
target = 'Age'
model_name = 'DenseNet121'
optimizer_name = 'Adam'
learning_rate = 0.01

#regularization: start with zero regularization. After good training performance AND overfitting is confirmed, use regularization.
lam=0.0 #regularization: weight shrinking
dropout_rate=0.0


In [2]:
###load libraries

#read and write
import os
import sys
import json
import pickle
import tarfile
import shutil
import pyreadr
import csv

#maths
import numpy as np
import pandas as pd
import math
import random
from math import sqrt
from numpy.polynomial.polynomial import polyfit

#images
import matplotlib.pyplot as plt
from PIL import Image
from scipy.ndimage import shift, rotate
from skimage.color import gray2rgb

#miscellaneous
import warnings
import multiprocessing as mp
from tqdm import tqdm_notebook as tqdm
import gc

#tensorflow
import tensorflow as tf
from tensorflow.keras.utils import Sequence
from tensorflow import set_random_seed

#sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, precision_recall_curve, average_precision_score
from sklearn.utils import class_weight

#keras
import keras
from keras import backend as K
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Flatten, Dense, Activation, Input, Reshape, BatchNormalization, InputLayer, Dropout, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Conv3D, MaxPooling3D, GlobalAveragePooling2D, LSTM
from keras.models import Sequential, Model, model_from_json, clone_model
from keras import regularizers, optimizers
from keras.optimizers import Adam, RMSprop, Adadelta
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, TerminateOnNaN, TensorBoard, ModelCheckpoint
from sklearn.metrics import r2_score, mean_squared_error
#from tensorflow.keras.metrics import Recall, Precision, AUC


Using TensorFlow backend.


In [3]:
#functions to put in helper file
def append_ext(fn):
    return fn+".jpg"

def generate_base_model(model_name, lam, dropout_rate, import_weights):
    if model_name in ['VGG16', 'VGG19']:
        if model_name == 'VGG16':
            from keras.applications.vgg16 import VGG16
            base_model = VGG16(include_top=False, weights=import_weights, input_shape=(224,224,3))
        elif model_name == 'VGG19':
            from keras.applications.vgg19 import VGG19
            base_model = VGG19(include_top=False, weights=import_weights, input_shape=(224,224,3))
        x = base_model.output
        x = Flatten()(x)
        x = Dense(4096, activation='relu', kernel_regularizer=regularizers.l2(lam))(x)
        x = Dropout(dropout_rate)(x)
        x = Dense(4096, activation='relu', kernel_regularizer=regularizers.l2(lam))(x)
        x = Dropout(dropout_rate)(x) 
    elif model_name in ['MobileNet', 'MobileNetV2']:
        if model_name == 'MobileNet':
            from keras.applications.mobilenet import MobileNet
            base_model = MobileNet(include_top=False, weights=import_weights, input_shape=(224,224,3))
        elif model_name == 'MobileNetV2':
            from keras.applications.mobilenet_v2 import MobileNetV2
            base_model = MobileNetV2(include_top=False, weights=import_weights, input_shape=(224,224,3))
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
    elif model_name in ['DenseNet121', 'DenseNet169', 'DenseNet201']:
        if model_name == 'DenseNet121':
            from keras.applications.densenet import DenseNet121
            base_model = DenseNet121(include_top=True, weights=import_weights, input_shape=(224,224,3))
        elif model_name == 'DenseNet169':
            from keras.applications.densenet import DenseNet169
            base_model = DenseNet169(include_top=True, weights=import_weights, input_shape=(224,224,3))
        elif model_name == 'DenseNet201':
            from keras.applications.densenet import DenseNet201
            base_model = DenseNet201(include_top=True, weights=import_weights, input_shape=(224,224,3))            
        base_model = Model(base_model.inputs, base_model.layers[-2].output)
        x = base_model.output
    elif model_name in ['NASNetMobile', 'NASNetLarge']:
        if model_name == 'NASNetMobile':
            from keras.applications.nasnet import NASNetMobile
            base_model = NASNetMobile(include_top=True, weights=import_weights, input_shape=(224,224,3))
        elif model_name == 'NASNetLarge':
            from keras.applications.nasnet import NASNetLarge
            base_model = NASNetLarge(include_top=True, weights=import_weights, input_shape=(331,331,3))
        base_model = Model(base_model.inputs, base_model.layers[-2].output)
        x = base_model.output
    elif model_name == 'Xception':
        from keras.applications.xception import Xception
        base_model = Xception(include_top=False, weights=import_weights, input_shape=(299,299,3))
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
    elif model_name == 'InceptionV3':
        from keras.applications.inception_v3 import InceptionV3
        base_model = InceptionV3(include_top=False, weights=import_weights, input_shape=(299,299,3))
        x = base_model.output        
        x = GlobalAveragePooling2D()(x)
    elif model_name == 'InceptionResNetV2':
        from keras.applications.inception_resnet_v2 import InceptionResNetV2
        base_model = InceptionResNetV2(include_top=False, weights=import_weights, input_shape=(299,299,3))
        x = base_model.output        
        x = GlobalAveragePooling2D()(x)
    return x, base_model.input

def complete_architecture(x, input_shape, lam, dropout_rate):
    x = Dense(1024, activation='selu', kernel_regularizer=regularizers.l2(lam))(x)
    x = Dropout(dropout_rate)(x)
    x = Dense(512, activation='selu', kernel_regularizer=regularizers.l2(lam))(x)
    x = Dropout(dropout_rate)(x)
    x = Dense(128, activation='selu', kernel_regularizer=regularizers.l2(lam))(x)
    x = Dropout(dropout_rate)(x)
    x = Dense(64, activation='selu', kernel_regularizer=regularizers.l2(lam))(x)
    x = Dropout(dropout_rate)(x)
    predictions = Dense(1, activation='linear')(x)
    model = Model(inputs=input_shape, outputs=predictions)
    return model

def R_squared(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred )) 
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) ) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )
  
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

def set_learning_rate(model, optimizer_name, learning_rate):
    opt = globals()[optimizer_name](lr=learning_rate)
    model.compile(optimizer=opt, loss='mean_squared_error', metrics=[R_squared, root_mean_squared_error])
    
def initialize_history():
    HISTORY = {}
    for metric in ['loss'] + metrics:
        for fold in folds_tune:
            if(fold=='train'):
                HISTORY[metric] = []
            else:
                HISTORY[fold + '_' + metric] = []
    return HISTORY

def update_history(HISTORY, history):
    keys = history.history.keys()
    for key in keys:
        HISTORY[key] = HISTORY[key] + history.history[key]
    return HISTORY
    
def plot_training(HISTORY, version):
    keys = history.history.keys()
    fig, axs = plt.subplots(1, int(len(keys)/2), sharey=False, sharex=True)
    fig.set_figwidth(15)
    fig.set_figheight(5)
    epochs = np.array(range(len(HISTORY[metrics[0]])))
    for i, metric in enumerate(['loss'] + metrics):
        for key in [key for key in keys if metric in key][::-1]:
            axs[i].plot(epochs, HISTORY[key])
        axs[i].legend(['Training ' + metric, 'Validation ' + metric])
        axs[i].set_title(metric + ' = f(Epoch)')
        axs[i].set_xlabel('Epoch')
        axs[i].set_ylabel(metric)
        axs[i].set_ylim((-0.2, 1.1))
    #save figure as pdf
    fig.savefig("../figures/Training_" + version + '.pdf', bbox_inches='tight')
    
def save_model_weights(model, version):
    model.save_weights(path_store + "model_weights_" + version + ".h5")
    print("Model's best weights for "+ version + " were saved.")
    
def rmse(y_true, y_pred):
    return sqrt(mean_squared_error(y_true, y_pred))

def generate_predictions_and_performances(model, GENERATORS, STEPSIZES):
    PREDS={}
    PERFORMANCES={}
    for fold in folds:
        generator = GENERATORS[fold]
        generator.reset()
        PREDS[fold]=model.predict_generator(generator, STEPSIZES[fold], verbose=1)
    for metric in metrics:
            PERFORMANCES[metric] = {}
            for fold in folds:
                PERFORMANCES[metric][fold] = metric_functions[metric](DATA_FEATURES[fold][target], PREDS[fold])
    return PREDS, PERFORMANCES


In [4]:
#parameters to put in helper file
folds = ['train', 'val', 'test']
folds_tune = ['train', 'val']
models_names = ['VGG16', 'VGG19', 'MobileNet', 'MobileNetV2', 'DenseNet121', 'DenseNet169', 'DenseNet201', 'NASNetMobile', 'NASNetLarge', 'Xception', 'InceptionV3', 'InceptionResNetV2']
images_sizes = ['224', '299', '331']
metrics = ['R_squared', 'root_mean_squared_error']
main_metrics = dict.fromkeys(['Age'], 'R_squared')
main_metrics.update(dict.fromkeys(['Sex'], 'AUC'))
metric_functions = {'R_squared':r2_score, 'root_mean_squared_error':rmse}

#define dictionary to resize the images to the right size depending on the model
input_size_models = dict.fromkeys(['VGG16', 'VGG19', 'MobileNet', 'MobileNetV2', 'DenseNet121', 'DenseNet169', 'DenseNet201', 'NASNetMobile'], 224)
input_size_models.update(dict.fromkeys(['Xception', 'InceptionV3', 'InceptionResNetV2'], 299))
input_size_models.update(dict.fromkeys(['NASNetLarge'], 331))

#define dictionaries to format the text
dict_folds={'train':'Training', 'val':'Validation', 'test':'Testing'}

#define paths
if '/Users/Alan/' in os.getcwd():
    os.chdir('/Users/Alan/Desktop/Aging/Medical_Images/scripts/')
    path_store = '../data/'
    path_compute = '../data/'
else:
    os.chdir('/n/groups/patel/Alan/Aging/Medical_Images/scripts/')
    path_store = '../data/'
    path_compute = '/n/scratch2/al311/Aging/Medical_Images/data/'

if image_type == 'Liver':
    dir_images = path_store + '../../../../uk_biobank/main_data_52887/Liver/Liver_20204/'
else:
    sys.exit("Error. Image type not available")

#model
image_size = input_size_models[model_name]
import_weights = 'imagenet' #choose between None and 'imagenet'

#compiler
batch_size = 64
n_epochs = 10
continue_training = True
main_metric = main_metrics[target]
version = target + '_' + image_type + '_' + model_name + '_' + optimizer_name + '_' + str(learning_rate) + '_' + str(lam) + '_' + str(dropout_rate) + '_' + str(batch_size)

#postprocessing
boot_iterations=10000

#set parameters
random.seed(0)
set_random_seed(0)

In [5]:
#print versions and info
print('tensorflow version : ', tf.__version__)
print('Build with Cuda : ', tf.test.is_built_with_cuda())
print('Gpu available : ', tf.test.is_gpu_available())
#print('Available ressources : ', tf.config.experimental.list_physical_devices())
config = tf.ConfigProto()
#device_count = {'GPU': 1, 'CPU': mp.cpu_count() },log_device_placement =  True)
config.gpu_options.allow_growth = True
sess= tf.Session(config = config)
K.set_session(session= sess)
K.tensorflow_backend._get_available_gpus()
warnings.filterwarnings('ignore')

tensorflow version :  1.13.1
Build with Cuda :  True
Gpu available :  True


In [6]:
#explore which variables are present in the features dataset
#data_features = pd.read_csv(path_store + "../../../../uk_biobank/main_data_52887/ukb37397.csv", nrows=1)
#data_features.columns.values

In [7]:
#load the selected features
data_features = pd.read_csv(path_store + "../../../../uk_biobank/main_data_52887/ukb37397.csv", usecols=['eid','21003-0.0','31-0.0', '22414-2.0'])
data_features.columns = ['eid', 'Sex','Age', 'Data_quality']
data_features['eid'] =  data_features['eid'].astype(str)
data_features = data_features.set_index('eid', drop=False)
#remove the samples for which the liver data is low quality
data_features = data_features[data_features['Data_quality']!=np.nan]
data_features = data_features.drop("Data_quality", axis=1)
#get rid of samples with NAs
data_features = data_features.dropna()
#list the samples' ids for which liver images are available
all_files = os.listdir(dir_images)
all_ids = [file.split(".", maxsplit=1)[0] for file in all_files]
data_features = data_features.loc[all_ids]
files = data_features.index.values
#save the features
data_features.to_csv(path_store + "data_features_" + version + ".csv", index=False)

In [8]:
#define data generators
datagen_train=ImageDataGenerator(rescale=1./255.)
datagen_test = ImageDataGenerator(rescale=1./255.)
class_mode_train = "raw"
class_mode_test = None

#generate ids and image generators for train, val, test
folds = ['train', 'val', 'test']
data_features = pd.read_csv(path_store + "data_features_" + version + ".csv")
data_features['eid'] = data_features['eid'].apply(str)
data_features["eid"] = data_features["eid"].apply(append_ext)
data_features.set_index('eid', drop=False)
ids = data_features.index.values.copy()
np.random.shuffle(ids)
percent_train = 0.8
percent_val = 0.1
n_discard = len(ids)%batch_size
n_limit_train = math.ceil(len(ids)/batch_size*percent_train)*batch_size
n_limit_val = math.ceil(len(ids)/batch_size*(percent_train+percent_val))*batch_size
n_limit_test = len(ids)-n_discard

#split IDs
IDs={}
IDs['train'] = ids[:n_limit_train]
IDs['val'] = ids[n_limit_train:n_limit_val]
IDs['test'] = ids[n_limit_val:n_limit_test]

#compute values for scaling of Age
fold = 'train'
idx = np.where(np.isin(data_features.index.values, IDs['train']))[0]
data_features_train = data_features.iloc[idx,:]
Age_mean = data_features_train['Age'].mean()
Age_std = data_features_train['Age'].std()

#split data_features
indices={}
DATA_FEATURES = {}
GENERATORS = {}
STEP_SIZES = {}
for fold in folds:
    indices[fold] = np.where(np.isin(data_features.index.values, IDs[fold]))[0]
    data_features_fold = data_features.iloc[indices[fold],:]
    data_features_fold.to_csv(path_store + "data_features_" + fold + ".csv")
    data_features_fold['Age'] = (data_features_fold['Age']-Age_mean)/Age_std
    if fold == 'test':
        datagen=datagen_test
        class_mode = class_mode_test
    else:
        datagen=datagen_train
        class_mode = class_mode_train
    
    #define data generator
    generator_fold = datagen.flow_from_dataframe(
        dataframe=data_features_fold,
        directory=dir_images,
        x_col="eid",
        y_col=target,
        color_mode="rgb",
        batch_size=batch_size,
        seed=0,
        shuffle=True,
        class_mode="raw",
        target_size=(image_size,image_size))
    
    #assign variables to their names
    DATA_FEATURES[fold] = data_features_fold
    GENERATORS[fold] = generator_fold
    STEP_SIZES[fold] = generator_fold.n//generator_fold.batch_size

#define the model
x, base_model_input = generate_base_model(model_name=model_name, lam=lam, dropout_rate=dropout_rate, import_weights=import_weights)
model = complete_architecture(x=x, input_shape=base_model_input, lam=lam, dropout_rate=dropout_rate)

#initialise history
HISTORY = initialize_history()

#take subset to debunk
#DATA_FEATURES['train'] = DATA_FEATURES['test']
#GENERATORS['train'] = GENERATORS['test']
#STEP_SIZES['train'] = STEP_SIZES['test']
#n_epochs = 2

Found 30336 validated image filenames.
Found 3776 validated image filenames.
Found 3712 validated image filenames.
Instructions for updating:
Colocations handled automatically by placer.


In [9]:
#(re-)set the learning rate
set_learning_rate(model, optimizer_name, learning_rate)

In [10]:
#load weights to continue training
path_weights = path_store + "model_weights_" + version + ".h5"
if continue_training & os.path.exists(path_weights):
    print("loading previous model's weights")
    #load weights
    model.load_weights(path_weights)
    #load previous best performance
    json_file = open(path_store + 'Performance_' + version + '.json', 'r')
    best_perf = json_file.read()
    json_file.close()
    N_epochs = best_perf['N_epochs']
    max_perf_val = best_perf[main_metric]['val']
else:
    N_epochs = 0
    max_perf_val = -np.Inf

In [ ]:
#train the model
while True:
    history = model.fit_generator(generator=GENERATORS['train'],
                    steps_per_epoch=STEP_SIZES['train'],
                    validation_data=GENERATORS['val'],
                    validation_steps=STEP_SIZES['val'],
                    use_multiprocessing = True,
                    epochs=n_epochs)
    #compute performances
    N_epochs += n_epochs
    HISTORY = update_history(HISTORY, history)
    plot_training(HISTORY, version)
    PREDS, PERF = generate_predictions_and_performances(model, GENERATORS, STEP_SIZES)
    print('N_epochs = ' + str(N_epochs))
    print('Performance summary: ')
    print(PERF)
    if np.max(history.history['val_' + main_metric]) > max_perf_val:
        print('A better model was found in the middle of the epoch batch with validation ' + main_metric + ' = ' + str(np.max(history.history['val_' + main_metric])))
    if np.max(PERF[main_metric]['val']) > max_perf_val:
        max_metric_val = np.max(PERF[main_metric]['val'])
        save_model_weights(model, version)
        to_save = PERF.copy()
        to_save['version'] = version
        to_save['N_epochs'] = N_epochs
        json.dump(to_save, open(path_store + 'Performance_' + version + '.json','w'))


Instructions for updating:
Use tf.cast instead.
Epoch 1/10
474/474 [==============================] - 637s 1s/step - loss: 63.1770 - R_squared: -61.1766 - root_mean_squared_error: 1.9518 - val_loss: 1.1078 - val_R_squared: -0.1328 - val_root_mean_squared_error: 1.0500
Epoch 2/10
474/474 [==============================] - 191s 404ms/step - loss: 0.8964 - R_squared: 0.0824 - root_mean_squared_error: 0.9372 - val_loss: 2.1768 - val_R_squared: -1.2105 - val_root_mean_squared_error: 1.4711
Epoch 3/10
474/474 [==============================] - 190s 400ms/step - loss: 0.8656 - R_squared: 0.1145 - root_mean_squared_error: 0.9211 - val_loss: 0.8045 - val_R_squared: 0.1835 - val_root_mean_squared_error: 0.8939
Epoch 4/10
474/474 [==============================] - 190s 400ms/step - loss: 0.7572 - R_squared: 0.2240 - root_mean_squared_error: 0.8651 - val_loss: 0.9469 - val_R_squared: 0.0414 - val_root_mean_squared_error: 0.9706
Epoch 5/10
474/474 [==============================] - 190s 401ms/step 